In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("Data/flipkart_product_review.csv")
data

In [ ]:
# !pip install -q langchain
# !pip install -q langchain-community
# !pip install -q langchain-astradb
# !pip install -q langchain-groq
# !pip install -q pypdf

In [ ]:
data.columns

In [ ]:
data = data[["product_title","review"]]
data.head()

In [ ]:
# Convert Data into Dcoument Fromat
# [ Document(metadata={'product_name': 'BoAt BassHeads 100 Wired Headset'}, page_content='a very good earphone in budget sound is awesome very pure sound but the bass is ok ok.. you can go for it without thinking much') ]

In [ ]:
product_list = []

## Itrate over the rows of the DataFrame

for index, row in data.iterrows():
  object = {
      "product_name": row["product_title"],
      "review": row["review"]
  }

  ## Append the object to the product list

  product_list.append(object)

In [ ]:
product_list

In [ ]:
from langchain_core.documents import Document

In [ ]:
docs = []

for object in product_list:
  metadata = {"product_name": object["product_name"]}
  page_content = object["review"]

  doc = Document(page_content= page_content, metadata= metadata)

  docs.append(doc)

In [ ]:
docs[0]

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API = os.getenv("GROQ_API")
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE = os.getenv("ASTRA_DB_KEYSPACE")
HF_TOKEN = os.getenv("HF_TOKEN")

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

In [ ]:
embeddings = HuggingFaceInferenceAPIEmbeddings(api_key= HF_TOKEN, model_name= "BAAI/bge-base-en-v1.5")

In [ ]:
from langchain_astradb import AstraDBVectorStore

In [ ]:
vstore = AstraDBVectorStore(
    embedding= embeddings,
    collection_name= "flipkart",
    api_endpoint = ASTRA_DB_API_ENDPOINT,
    token = ASTRA_DB_APPLICATION_TOKEN,
    namespace = ASTRA_DB_KEYSPACE

)

In [ ]:
insert_ids = vstore.add_documents(docs)

In [ ]:
from langchain_groq import ChatGroq

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

In [ ]:
model = ChatGroq(groq_api_key = GROQ_API, model="deepseek-r1-distill-llama-70b", temperature=0.5)

In [ ]:

retriever_prompt = ("Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
    )

In [ ]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", retriever_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(model, retriever, contextualize_q_prompt)

In [ ]:
PRODUCT_BOT_TEMPLATE = """
    Your ecommercebot bot is an expert in product recommendations and customer queries.
    It analyzes product titles and reviews to provide accurate and helpful responses.
    Ensure your answers are relevant to the product context and refrain from straying off-topic.
    Your responses should be concise and informative.

    CONTEXT:
    {context}

    QUESTION: {input}

    YOUR ANSWER:

    """

In [ ]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", PRODUCT_BOT_TEMPLATE),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

In [ ]:
chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
chat_history = []
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

In [ ]:
def get_session_history(session_id: str)-> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id]= ChatMessageHistory()
  return store[session_id]

In [ ]:
chain_with_memmory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
chain_with_memmory.invoke(
   {"input": "can you tell me the best bluetooth buds?"},
    config={
        "configurable": {"session_id": "sohan"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
store